In [1]:
import ecole
import shutil
import os
import sys
import time
import numpy as np
from tqdm import trange
import gurobipy as gb
import pyscipopt as pyscip
from gurobipy import GRB

scip_parameters = {"limits/time": 360}

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout


In [18]:
#scip
model_scip = pyscip.scip.Model()
model_scip.readProblem(filename="debug2.lp",extension = "lp")
model_scip.optimize()
time.sleep(1)
print("\n",model_scip.getBestSol())

# ecole
print("Ecole*********************")
with HiddenPrints():
    solver = ecole.scip.Model.from_file("debug2.lp")
    solver.solve()
    aspyscip = solver.as_pyscipopt()
    sol = aspyscip.getBestSol()
print(sol)

#gurobi
print("Gurobi*********************")
with HiddenPrints():
    model = gb.read("debug2.lp")
    model.optimize()
for var in model.getVars():
    print(f"{var.varName}: {round(var.X, 3)}")



original problem has 4 variables (4 bin, 0 int, 0 impl, 0 cont) and 4 constraints
feasible solution found by trivial heuristic after 0.0 seconds, objective value 2.510000e+02
presolving:
(round 1, fast)       1 del vars, 1 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 3 clqs
   (0.0s) running MILP presolver
   (0.0s) MILP presolver found nothing
(round 2, exhaustive) 1 del vars, 1 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 3 upgd conss, 0 impls, 3 clqs
   (0.0s) probing cycle finished: starting next cycle
   (0.0s) symmetry computation started: requiring (bin +, int -, cont +), (fixed: bin -, int +, cont -)
   (0.0s) no symmetry present
presolving (3 rounds: 3 fast, 2 medium, 2 exhaustive):
 1 deleted vars, 1 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 3 cliques
presolved problem has 3 variables (3 bin, 0 int, 0 impl, 0 cont) and 3 co

In [3]:
setCover = ecole.instance.SetCoverGenerator\
            (n_rows=300, n_cols=400,density=0.1)
env = ecole.environment.Branching(observation_function = ecole.observation.NodeBipartite())
instance = next(setCover)
print(instance)
problme_name = "set_cover_test"
instance.write_problem(problme_name+".lp")

obs, action_set, _, _, _ = env.reset(instance)
print(obs.row_features.shape)
print(obs.edge_features.shape)
print(obs.variable_features.shape)


(337, 5)
[337, 77]
(77, 19)


# Dump features in file Json

In [2]:
import json
def dumpRowFeatures(filename,row_features):
    features = {}
    num = 0
    for node in row_features:
        node_features = {"bias" : node[0],
                    "objective_cosine_similarity" : node[1],
                   "is_tight" : node[2],
                   "dual_solution_value" : node[3],
                   "scaled_age" : node[4]
                    }
        features[num] = node_features
        num+=1
    data = json.dumps(features)
    file = open(filename,'w')
    file.write(data)
    file.close()

def dumpVariableFeatures(filename,variable_features):
    features = {}
    num = 0
    for node in variable_features:
        node_features = {"objective" : node[0],
                "is_type_binary" : node[1],
               "is_type_integer" : node[2],
               "is_type_implicit_integer" : node[3],
               "is_type_continuous" : node[4],
                "has_lower_bound" : node[5],
                "has_upper_bound" : node[6],
               "normed_reduced_cost" : node[7],
               "solution_value" : node[8],
               "solution_frac" : node[9],
                "is_solution_at_lower_bound" : node[10],
                "is_solution_at_upper_bound" : node[11],
               "scaled_age" : node[12],
               "incumbent_value" : node[13],
               "average_incumbent_value" : node[14],
                "is_basis_lower" : node[15],
                "is_basis_basic" : node[16],
               "is_basis_upper" : node[17],
               "is_basis_zero" : node[18]
                }
        features[num] = node_features
        num+=1
    data = json.dumps(features)
    file = open(filename,'w')
    file.write(data)
    file.close()

def dumpSolution_Ecole(filename,solver):
    pyscip = solver.as_pyscipopt()
    bestsol = pyscip.getBestSol()
    solutions = {
        "Best_Solution":bestsol.__str__(),
        "Solution_Pool":pyscip.getSols().__str__()
    }
    data = json.dumps(solutions)
    file = open(filename,'w')
    file.write(data)
    file.close()

def dumpSolution_Gurobi(filename,solver,nbSolMax = 10,poolGap=0.1):
    # Limit how many solutions to collect
    model.setParam(GRB.Param.PoolSolutions, nbSolMax)

    # Limit the search space by setting a gap for the worst possible solution
    # that will be accepted
    model.setParam(GRB.Param.PoolGap, poolGap)

    # do a systematic search for the k- best solutions
    model.setParam(GRB.Param.PoolSearchMode, 2)

    model.optimize()
    status = model.Status
    if status in (GRB.INF_OR_UNBD, GRB.INFEASIBLE, GRB.UNBOUNDED):
        print('The model cannot be solved because it is infeasible or unbounded')
        sys.exit(1)
    if status != GRB.OPTIMAL:
        print('Optimization was stopped with status ' + str(status))
        sys.exit(1)
      #the first solution is the best  
#     #get the beat solution
#     bestSol = {}
#     for var in model.getVars():
#         bestSol[var.varName] = var.X
        

    #get the solution pool
    poolSol = {}
    nSolution = model.SolCount
    for e in range (nSolution):
        sol = {}
        model.setParam(GRB.Param.SolutionNumber, e)
        for var in model.getVars():
            sol[var.varName] = var.X
        poolSol[model.PoolObjVal]=sol
    
    solutions = {
        "Nombre_Solutions":nSolution,
#         "Best_Solution":bestSol,
        "Solution_Pool":poolSol
    }

    data = json.dumps(solutions)
    file = open(filename,'w')
    file.write(data)
    file.close()
    

# dumpRowFeatures("features/row.json",obs.row_features)
# dumpVariableFeatures("features/v.json",obs.variable_features)
# model = gb.read(problme_name+".lp")

# model.optimize()
# dumpSolution_Gurobi("test_dump_gurobi.json",model)

# Generate Problem and collect features

In [5]:
# if os.path.exists("problem"):
#     shutil.rmtree("problem/")
# os.mkdir("problem/")
# if os.path.exists("features"):
#     shutil.rmtree("features/")
# os.mkdir("features/")

env = ecole.environment.Branching(observation_function = ecole.observation.NodeBipartite(),scip_params = scip_parameters)
# print(obs.row_features.shape)
# print(obs.edge_features.shape)
# print(obs.variable_features.shape)

for row in [100,150,200]:
    for coef_col in [1,1.5,2]: 
        for d in [0.1,0.15,0.2]:
            col = int(coef_col * row)
            setCover = ecole.instance.SetCoverGenerator(n_rows = row, n_cols = col,density = d) 
            print("Generate with Row:%d,Col:%d,Density:%f" % (row,col,d))
            for n in trange(1,101):
                done = False
                while(not done):
                    try:
                        instance = next(setCover)
                        #save problm lp
                        problme_name = \
                            "set_cover_{"+row.__str__()+"*"+col.__str__()+"_"+d.__str__()+"_"+n.__str__()+"}"
                        instance.write_problem("problem/"+problme_name+".lp")
                        #save features
                        obs, _, _, _, _ = env.reset(instance)
                        dumpRowFeatures("features/"+problme_name+"_row_features.json",obs.row_features)
                        dumpVariableFeatures("features/"+problme_name+"_variable_features.json",obs.variable_features)
                        #save label
    #                             solver = ecole.scip.Model.from_file("problem/"+problme_name+".lp")
    #                             solver.solve()
    #                             dumpSolution("label/"+problme_name+"_label.json",solver)
                        done = True
                    except:
                        done = False
            

Generate with Row:100,Col:100,Density:0.100000


100%|█████████████████████████████████████████| 100/100 [01:11<00:00,  1.40it/s]


Generate with Row:100,Col:100,Density:0.150000


100%|█████████████████████████████████████████| 100/100 [01:24<00:00,  1.18it/s]


Generate with Row:100,Col:100,Density:0.200000


...

100%|█████████████████████████████████████████| 100/100 [02:47<00:00,  1.67s/it]


Generate with Row:200,Col:400,Density:0.150000


100%|█████████████████████████████████████████| 100/100 [02:33<00:00,  1.53s/it]


Generate with Row:200,Col:400,Density:0.200000


100%|█████████████████████████████████████████| 100/100 [02:32<00:00,  1.52s/it]


# Collect labels

In [10]:
# solver = ecole.scip.Model.from_file("debug.lp")
# solver.solve()
# print(solver.is_solved)
# # solver.write_problem("debug_w")
# pyscip = solver.as_pyscipopt()
# sol = pyscip.getBestSol()
# print(sol)

if os.path.exists("label"):
    shutil.rmtree("label/")
os.mkdir("label/")
for row in [100,150,200]:
    for coef_col in [1,1.5,2]: 
        for d in [0.1,0.15,0.2]:
            col = int(coef_col * row)
            print("Solve with Row:%d,Col:%d,Density:%f" % (row,col,d))
            for n in trange(1,101):
#                 #save label
                with HiddenPrints():
                    problme_name = \
                            "set_cover_{"+row.__str__()+"*"+col.__str__()+"_"+d.__str__()+"_"+n.__str__()+"}"
                    solver = gb.read("problem/"+problme_name+".lp")
                    solver.optimize()
    #                 solver = ecole.scip.Model.from_file("problem/"+problme_name+".lp")
    #                 solver.solve()
                    dumpSolution_Gurobi("label/"+problme_name+"_label.json",solver)

Solve with Row:100,Col:100,Density:0.100000


100%|█████████████████████████████████████████| 100/100 [00:04<00:00, 24.82it/s]


Solve with Row:100,Col:100,Density:0.150000


100%|█████████████████████████████████████████| 100/100 [00:04<00:00, 20.66it/s]


Solve with Row:100,Col:100,Density:0.200000


...

100%|█████████████████████████████████████████| 100/100 [00:12<00:00,  8.02it/s]


Solve with Row:200,Col:400,Density:0.150000


100%|█████████████████████████████████████████| 100/100 [00:14<00:00,  6.97it/s]


Solve with Row:200,Col:400,Density:0.200000


100%|█████████████████████████████████████████| 100/100 [00:16<00:00,  6.10it/s]
